In [1]:
# Project Libraries
import data
import functions as fn

In [2]:
# Generic Libraries
import numpy as np
import pandas as pd

In [3]:
# Position valuation
def other(size_variation: float, price_variation: float, operation: str = None, constant_position_base: float = .0001):
    # Ask
    if operation == 'ask':
        if price_variation > 0:
            current_position = constant_position_base
        elif price_variation == 0:
            if size_variation > 0:
                current_position = 0
            else:
                if size_variation <= constant_position_base:
                    current_position = size_variation
                else:
                    current_position = constant_position_base
        else:
            current_position = 0
    # Bid
    if operation == 'bid':
        if price_variation < 0:
            current_position = constant_position_base
        elif price_variation == 0:
            if size_variation > 0:
                current_position = 0
            else:
                if size_variation <= constant_position_base:
                    current_position = size_variation
                else:
                    current_position = constant_position_base
        else:
            current_position = 0
    return current_position

In [7]:
# ---- Rebalance
def rebalance(trade_fee, df_trades, df_inventory, df_rebalance, rebalance_index):
    
    decimals_usdt=10e18
    decimals_btc=10e8
    decimals_btcusdt=int(decimals_btc/decimals_usdt)
    decimals_usdtbtc=int(decimals_usdt/decimals_btc)
    initial_base = df_inventory.at[0, 'base (BTC)']
    last_base = df_inventory.at[rebalance_index - 1, 'base (BTC)']
    initial_quote = df_inventory.at[0, 'quote (USDT)']
    last_quote = df_inventory.at[rebalance_index - 1, 'quote (USDT)']

    inventory_index = df_inventory.index[-1] + 1

    if df_inventory['base (BTC)'].iloc[-1] < 4.5 :
        swap_price = decimals_usdtbtc/(((int(np.sqrt(df_trades['price'].iloc[0])*96)*2)/(2*96))*2)
        swap_amount = initial_base - last_base
        swap_fee = trade_fee * decimals_usdtbtc/swap_price * swap_amount

        df_rebalance.at[rebalance_index,'timestamp'] = df_inventory['timestamp'].iloc[-1]
        df_rebalance.at[rebalance_index,'sent'] = 'USDT'
        df_rebalance.at[rebalance_index,'recieved'] = 'BTC'
        df_rebalance.at[rebalance_index,'price'] = swap_price
        df_rebalance.at[rebalance_index,'fee'] = swap_fee

        fee = decimals_usdtbtc/(int([ swap_fee ])/(2*96))*2

        df_inventory.at[inventory_index, 'timestamp'] = df_inventory.at[inventory_index - 1, 'timestamp']
        df_inventory.at[inventory_index, 'base (BTC)'] = last_base - fee + decimals_usdtbtc/(int([ swap_price ])/(2*96))*2 * swap_amount


    elif df_inventory['quote (USDT)'].iloc[-1] < 499980:
        swap_price = decimals_btcusdt/(((int(np.sqrt(df_trades['price'].iloc[0])*96)*2)/(2*96))*2)
        swap_amount = initial_quote - last_quote
        swap_fee = trade_fee * decimals_btcusdt/swap_price * swap_amount

        df_rebalance.at[rebalance_index,'timestamp'] = df_inventory['timestamp'].iloc[-1]
        df_rebalance.at[rebalance_index,'sent'] = 'BTC'
        df_rebalance.at[rebalance_index,'recieved'] = 'USDT'
        df_rebalance.at[rebalance_index,'price'] = swap_price
        df_rebalance.at[rebalance_index,'fee'] = trade_fee*swap_price

        fee = decimals_usdtbtc/(int([ swap_fee ])/(2*96))*2

        df_inventory.at[inventory_index, 'timestamp'] = df_inventory.at[inventory_index - 1, 'timestamp']
        df_inventory.at[inventory_index, 'base (BTC)'] = last_base - fee 
        df_inventory.at[inventory_index, 'quote (USDT)'] = decimals_usdtbtc/(int([ swap_price ])/(2*96))*2 * swap_amount + last_quote

In [5]:
# ----- Market information
df_bitfinex_tob = pd.DataFrame(data.read_file(file_name = "orderbooks_05jul21.json", folder_route = "files/")['bitfinex']).transpose().reset_index()
df_bitfinex_tob['index'] = pd.to_datetime(df_bitfinex_tob['index'])
df_bitfinex_tob = df_bitfinex_tob.set_index('index').resample('S').last().ffill()
for column in df_bitfinex_tob.columns:
    df_bitfinex_tob[column] = df_bitfinex_tob[column].apply(lambda x: x['0'])
df_bitfinex_tob = df_bitfinex_tob.reset_index().rename(columns = {'index':'timestamp'})

# SLA Conditions
contracted_volume = .0001
platform_fee = .000025

# Initial Conditions Data Frames
df_info, df_inventory, df_rebalance = fn.dataframes()

# Initial Order
df_orders = pd.DataFrame([
    [df_bitfinex_tob.iloc[0]['timestamp'], df_bitfinex_tob.iloc[0]['timestamp']],
    [1,2], ['buy','sell'], [df_bitfinex_tob.iloc[0]['bid'], df_bitfinex_tob.iloc[0]['ask']],
    [contracted_volume, contracted_volume], ['BTC/USDT','BTC/USDT']
    ], 
    index =  ['timestamp','order_id','side','price', 'order_amount','symbol'],
    columns = [0,1]
).transpose()

for side_filled in ['ask','bid']:
    filled = fn.filled_volume(
        size_variation = df_bitfinex_tob.iloc[1][side_filled+'_size'] - df_bitfinex_tob.iloc[0][side_filled+'_size'],
        operation = side_filled, 
        price_variation = df_bitfinex_tob.iloc[1][side_filled] - df_orders[df_orders['side'] == fn.names(side_filled)].iloc[-1]['price'], 
        constant_position_base = contracted_volume
    )
    if filled > 0:            
            df_trades = fn.execute_trade(
                df_bitfinex_tob = df_bitfinex_tob,
                df_orders = df_orders,
                side = side_filled,
                i = 1,
                passed_index = 0, 
                current_id = 1, 
                traded_volume = filled,
                trade_fee = platform_fee,
                first_input = True)
            
            # ---- Liquidity Pool Adjustment
            passed_index_inventory = df_inventory.index[-1] + 1
            fn.inventory(
                 index_inventory = passed_index_inventory,
                 df_trades = df_trades, 
                 df_inventory = df_inventory
                 )

            if len(df_rebalance) == 0:
                index_rebalance = 0
            else:
                index_rebalance = df_rebalance.index[-1] + 1
            rebalance(
                trade_fee = platform_fee,
                rebalance_index = index_rebalance,
                df_inventory = df_inventory, 
                df_rebalance = df_rebalance,
                df_trades = df_trades
                )
            # ----

            last_id = df_orders.iloc[-1]['order_id']
            fn.place_order(
                i = 1,
                df_bitfinex_tob = df_bitfinex_tob,
                df_orders = df_orders,
                passed_index = 2,
                side = side_filled,
                current_id = last_id + 1, 
                amount = contracted_volume
                )
            
for period in range(2,len(df_bitfinex_tob)):
    for side_filled in ['ask','bid']:
        filled = fn.filled_volume(
            size_variation = df_bitfinex_tob.iloc[period][side_filled+'_size'] - df_bitfinex_tob.iloc[period - 1][side_filled+'_size'],
            operation = side_filled, 
            price_variation = df_bitfinex_tob.iloc[period][side_filled] - df_orders[df_orders['side'] == fn.names(side_filled)].iloc[-1]['price'], 
            constant_position_base = contracted_volume
        )
        if filled > 0:
            try:
                last_id_trades = df_trades.iloc[-1]['trade_id']
                passed_index_trades = df_trades.index[-1] + 1
                fn.execute_trade(
                    df_bitfinex_tob = df_bitfinex_tob,
                    df_trades = df_trades,
                    df_orders = df_orders,
                    side = side_filled,
                    i = period,
                    passed_index = passed_index_trades, 
                    current_id = last_id_trades + 1, 
                    traded_volume = filled, 
                    trade_fee = platform_fee
                    )
            except:
                df_trades = fn.execute_trade(
                df_bitfinex_tob = df_bitfinex_tob,
                df_orders = df_orders,
                side = side_filled,
                i = 1,
                passed_index = 0, 
                current_id = 1, 
                traded_volume = filled,
                trade_fee = platform_fee,
                first_input = True)
            
            # ---- Liquidity Pool Adjustment
            passed_index_inventory = df_inventory.index[-1] + 1
            fn.inventory(
                 index_inventory = passed_index_inventory,
                 df_trades = df_trades, 
                 df_inventory = df_inventory
                 )
            
            if len(df_rebalance) == 0:
                index_rebalance = 0
            else:
                index_rebalance = df_rebalance.index[-1] + 1
            rebalance(
                trade_fee = platform_fee,
                rebalance_index = index_rebalance,
                df_inventory = df_inventory, 
                df_rebalance = df_rebalance,
                df_trades = df_trades
                )
            # ----

            last_id_orders = df_orders.iloc[-1]['order_id']
            passed_index_orders = df_orders.index[-1] + 1
            fn.place_order(
                i = period,
                df_bitfinex_tob = df_bitfinex_tob,
                df_orders = df_orders,
                passed_index = passed_index_orders,
                side = side_filled,
                current_id = last_id_orders + 1, 
                amount = contracted_volume
                )

In [6]:
df_inventory

,timestamp,base (BTC),quote (USDT)
0,0.0,5.0000,500000.0000
1,2021-07-05 13:06:47+00:00,5.0001,499997.1725
2,2021-07-05 13:06:55+00:00,5.0002,499994.3447
3,2021-07-05 13:07:00+00:00,5.0003,499991.5164
4,2021-07-05 13:07:04+00:00,5.0004,499988.6867
5,2021-07-05 13:07:45+00:00,5.0005,499985.8563
6,2021-07-05 13:07:54+00:00,5.0006,499983.0258
7,2021-07-05 13:07:58+00:00,5.0007,499980.1943
8,2021-07-05 13:09:42+00:00,5.0008,499977.3618
9,2021-07-05 13:09:55+00:00,5.0009,499974.5288
